In [1]:
import numpy as np
import itertools as it

# Define Gear Properties

In [2]:
P = 48 #Pitch [teeth/in]
phi = np.pi*(20/180) #Pressure angle [rads]
b = (3/16) #Face width [in]
max_gears = 4 #only allow four gears to keep the gearbox compact
min_total_gear_ratio = 11.75
max_total_gear_ratio = 12.25
min_teeth = 18
max_teeth = 200
possible_gears = [ N for N in range (min_teeth, max_teeth) if N/P < 2.0 ] #Number of teeth all possible gears that fit size constraint (Pd < 2.0 in)
Qv = 10 #Quality Factor
Km = 1.3 #Mounting Factor assuming precision bearings and mountings
Ko = 1.5 #Overload Factor
motor_torque = 3.5 #[lb-f*in]
J = 4*[0.2] #worst case scenario Lewis form factor for all gears
Su = 90*1000 #Ultimate tensile stress[psi]

# Get all possible combinations of two stages

In [3]:
permutations = it.permutations(possible_gears, max_gears)
possible_gear_orderings = [tuple(gear_ordering) for gear_ordering in permutations]#convert to list of tuples

In [4]:
gear_orderings_with_desired_ratio = []
for gear_ordering in possible_gear_orderings:
    gear_ratio_1 = gear_ordering[1]/gear_ordering[0]
    gear_ratio_2 = gear_ordering[3]/gear_ordering[2]
    if (gear_ratio_1 > 1 and gear_ratio_2 > 1 and gear_ratio_1*gear_ratio_2 > min_total_gear_ratio
    and gear_ratio_1*gear_ratio_2 < max_total_gear_ratio):
        gear_orderings_with_desired_ratio.append(gear_ordering)

# Compute base diameters & radii and pitch diameters & radii

In [5]:
possible_gear_pitch_diameters = [(gear_ordering[0]/P, gear_ordering[1]/P, gear_ordering[2]/P, gear_ordering[3]/P)
                                 for gear_ordering in gear_orderings_with_desired_ratio ] #pitch diameters for all possible gears [in]
possible_gear_pitch_radii = [(Pd[0]/2, Pd[1]/2, Pd[2]/2, Pd[3]/2) for Pd in possible_gear_pitch_diameters] #pitch diameters for all possible gears [in]
possible_gear_base_radii = [(rp[0]*np.cos(phi), rp[1]*np.cos(phi), rp[2]*np.cos(phi), rp[3]*np.cos(phi))
                             for rp in possible_gear_pitch_radii] #base radii for all possible gears [in]

# Filter out all gear orderings that don't leave sufficient space for assembly

In [6]:
length = len(possible_gear_pitch_radii)
i = 0
while i < length:
    gear_ordering_idx = i
    gear_ordering = possible_gear_pitch_radii[i]
    r1 = gear_ordering[0]
    r2 = gear_ordering[1]
    r3 = gear_ordering[2]
    r4 = gear_ordering[3]
    i+=1
    if (r3+r4 <= r2):
        del possible_gear_pitch_radii[gear_ordering_idx]
        del possible_gear_pitch_diameters[gear_ordering_idx]
        del possible_gear_base_radii[gear_ordering_idx]
        del gear_orderings_with_desired_ratio[gear_ordering_idx]
        length -= 1

# Compute Vt and Kv

In [7]:
B = 0.25*(12-Qv)**(2/3)
A = 50+56*(1-B)
RPM = 5000 #[revs/min]
Vt = lambda rp: RPM*2*np.pi*rp/12
possible_gear_tangential_velocities = [(Vt(rp[0]), Vt(rp[1]), Vt(rp[2]), Vt(rp[3])) for rp in possible_gear_pitch_radii] #[ft/min]
Kv = lambda Vt: ((A + np.sqrt(Vt))/A)**B
possible_gear_Kvs = [(Kv(Vt[0]), Kv(Vt[1]), Kv(Vt[2]), Kv(Vt[3])) for Vt in possible_gear_tangential_velocities]                                         

# Compute Contact Ratio

In [8]:
Pb = np.pi*np.cos(phi)/P #base pitch
C = lambda rp1, rp2: rp1 + rp2 #center distance between two gears
possible_gear_center_distances = [(C(rp[0],rp[1]), C(rp[2], rp[3])) for rp in possible_gear_pitch_radii]
ra = lambda N: (N+2)/(2*P)
possible_gear_addendum_radii = [(ra(N[0]), ra(N[1]), ra(N[2]), ra(N[3])) for N in gear_orderings_with_desired_ratio]
Z = lambda ra1, ra2, rb1, rb2, C: np.sqrt(ra1**2 - rb1**2)+np.sqrt(ra2**2 - rb2**2) - C*np.sin(phi) #Contact Ratio numerator
CR = []
for i in range(len(possible_gear_addendum_radii)):
    ra = possible_gear_addendum_radii[i]
    rb = possible_gear_base_radii[i]
    center_distance_1 = C(rb[0], rb[1])
    numerator_1 = Z(ra[0], ra[1], rb[0], rb[1], center_distance_1)
    center_distance_2 = C(rb[2], rb[3])
    numerator_2 = Z(ra[2], ra[3], rb[2], rb[3], center_distance_2)
    CR1 = numerator_1/Pb
    CR2 = numerator_2/Pb
    CR.append((CR1, CR2))                         

# Compute Lewis AGMA Adjusted Bending Stresses for Gears 1 & 2

In [9]:
tangential_force_gear_1_and_2 = [motor_torque/(rp1[0]) for rp1 in possible_gear_pitch_radii] #[lb-f]
sigma_b = []
for i in range(len(tangential_force_gear_1_and_2)):
    Ft = tangential_force_gear_1_and_2[i]
    Kv_ = possible_gear_Kvs[i]
    sigma_b_1 = ((Ft*P)/(b*J[0]))*Km*Ko*Kv_[0]#bending stress [psi]
    sigma_b_2 = ((Ft*P)/(b*J[1]))*Km*Ko*Kv_[1]#bending stress [psi]
    sigma_b.append((sigma_b_1, sigma_b_2))

# Compute Lewis AGMA Adjusted Bending Stresses for Gears 3 & 4

In [10]:
tangential_force_gear_3_and_4 = [motor_torque*(rp3[1]/rp3[0])/(rp3[2]) for rp3 in possible_gear_pitch_radii] #[lb-f]
for i in range(len(tangential_force_gear_3_and_4)):
    Ft = tangential_force_gear_3_and_4[i]
    Kv_ = possible_gear_Kvs[i]
    sigma_b_3 = ((Ft*P)/(b*J[2]))*Km*Ko*Kv_[2]#bending stress [psi]
    sigma_b_4 = ((Ft*P)/(b*J[3]))*Km*Ko*Kv_[3]#bending stress [psi]
    sigma_b[i]= sigma_b[i]+(sigma_b_3, sigma_b_4)

# Life time stress analysis

In [11]:
Kr = 0.753
Cs = 0.85
Cf = 1
Ks = 1
Cl = 1
Cg = 1
Kt = 1 #moderate temperatures
Kms = 1.4 #non-idler
Sn_to_Su_ratio = 0.5 #infinite lifetime
Snprime = Sn_to_Su_ratio* Su #Allowable fatigue stress
Sn = Snprime*Cl*Cg*Cs*Kt*Kr*Kms #Maximum allowable stress [psi]

In [12]:
Sn

40323.149999999994

# Pick gear combination whose stresses are farthest under max stress

In [17]:
min = np.sum(sigma_b[0])/4
best_combo = sigma_b[0]
best_idx = 0
for idx in range(1, len(sigma_b)):
    gear_stress = sigma_b[idx]
    obj = 1*np.sum(gear_stress)/4
    if obj < min:
        best_combo = gear_orderings_with_desired_ratio[idx]
        best_idx = idx

In [18]:
best_combo

(42, 95, 18, 94)

# Manually enter actual Lewis form factors and compute actual stresses

In [21]:
J = [0.435, 0.45, 0.33, 0.43]
final_stresses = []
for i in range(len(J)):
    Kv_ = possible_gear_Kvs[best_idx]
    if i < 2:
        Ft = tangential_force_gear_1_and_2[best_idx]
    else:
        Ft = tangential_force_gear_3_and_4[best_idx]
    gear_stress = ((Ft*P)/(b*J[i]))*Km*Ko*Kv_[i] #[psi]
    final_stresses.append(gear_stress)

# Return final gear combination properties

In [29]:
final_pitch_radii = possible_gear_pitch_radii[best_idx]
print("best combination = ", best_combo)
print("Lewis AGMA stresses = ", final_stresses)
print("final pitch radii = ", possible_gear_pitch_radii[best_idx])
print("final gear ratio = ", (final_pitch_radii[1]/final_pitch_radii[0])*(final_pitch_radii[3]/final_pitch_radii[2]))
print("final outer radius = ", possible_gear_addendum_radii[best_idx])
print("final base radius = ", possible_gear_base_radii[best_idx])
print("final contact ratios = ", CR[best_idx])
print("final pitch diameters = ", possible_gear_pitch_diameters[best_idx])
print("final tangential tooth forces = ", (tangential_force_gear_1_and_2[best_idx], tangential_force_gear_3_and_4[best_idx]))

best combination =  (42, 95, 18, 94)
Lewis AGMA stresses =  [10503.99720414307, 10714.440815758875, 70104.042978834608, 59131.730397538275]
final pitch radii =  (0.4375, 0.9895833333333334, 0.1875, 0.9791666666666666)
final gear ratio =  11.812169312169312
final outer radius =  (0.4583333333333333, 1.0104166666666667, 0.20833333333333334, 1.0)
final base radius =  (0.41111552159383491, 0.92990415598605525, 0.17619236639735783, 0.92011569118620196)
final contact ratios =  (2.2634392004061343, 2.0789057540517577)
final pitch diameters =  (0.875, 1.9791666666666667, 0.375, 1.9583333333333333)
final tangential tooth forces =  (8.0, 42.22222222222222)
